第五章 一等函数

5.1 把函数当对象

In [1]:
def factoria(n):
    '''
    return:  n!
    '''
    return 1 if n<2 else n*factoria(n-1)

In [2]:
factoria.__doc__

'\n    return:  n!\n    '

In [3]:
type(factoria)

function

In [4]:
help(factoria)

Help on function factoria in module __main__:

factoria(n)
    return:  n!



函数是一等对象。可以把factorial函数赋值给变量fact，然后通过变量名调用。

In [5]:
fact = factoria
fact

<function __main__.factoria>

In [6]:
fact(5)

120

In [7]:
map(fact,range(11))

In [8]:
list(map(fact,range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

5.2高阶函数

接受函数为参数，或者把函数作为结果返回的函数叫做高阶函数。如map，sorted(可选的key参数用于提供一个函数）

In [9]:
fruits=['strawberry','fig','apple','cherry','raspberry','banana']
sorted(fruits)

['apple', 'banana', 'cherry', 'fig', 'raspberry', 'strawberry']

In [10]:
sorted(fruits,key=len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

In [11]:
#根据反向拼写给单词排序
def reverse(word):
    return word[::-1]
reverse('testing')

'gnitset'

In [12]:
print(reversed('testing'))
for i in reversed('testing'):
    print(i)


g
n
i
t
s
e
t


In [13]:
sorted(fruits,key=reverse)

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

由于引入了列表推导和生成器表达式，map和filter这两个内置函数已经不那么重要了，列表推导和生成器表达式更易于阅读

In [14]:
list(map(fact,range(6)))

[1, 1, 2, 6, 24, 120]

In [15]:
[fact(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [17]:
list(map(factoria,filter(lambda n:n%2,range(6))))

[1, 6, 120]

In [18]:
[fact(n) for n in range(6) if n%2]

[1, 6, 120]

在python3中，map和filter返回生成器，因此它们的直接替代品是生成器表达式（），python2中，返回的是列表，最接近的替代品是列表推导

python2中，reduce是内置函数，但是在python3中放到了functools模块里了。这个函数常用语求和，另一个内置的求和函数为sum，更好用。

In [19]:
from functools import reduce
from operator import add
reduce(add,range(100))

4950

In [20]:
sum(range(100))

4950

sum和reduce的通用思想是把某个操作连续应用到序列的元素上，累计之前的结果，把一系列值归约成一个值。

all和any也是内置的归约函数。

In [21]:
all(range(5))

False

In [22]:
any(range(5))

True

5.3 匿名函数

Python简单的句法限制了lambda函数的定义体只能使用纯表达式。也就是说lambda函数的定义中体不能赋值，也不能使用while和try等python语句

In [23]:
sorted(fruits,key=lambda word:word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

除了作为参数传给高阶函数之外，python很少使用匿名函数。由于句法上的限制，非平凡的lambda表达式要么难以阅读，要么无法写出。

5.4 可调用对象

除了用户自定义的函数，调用运算符（即()）还可以应用到其他对象上。callable()函数可判断对象是否可调用。python数据模型文档列出了7种可调用对象。

1.用户自定义的函数：用def或lambda表达式创建

![chapter5-4](image/chapter5-4.png)

In [24]:
abs,str,14

(<function abs>, str, 14)

In [25]:
[callable(obj) for obj in (abs,str,13)]

[True, True, False]

5.5 用户定义的可调用类型

不仅python函数是真正的对象，任何python对象都可以表现的像函数。为此，只需实现实例方法__call__

In [26]:
#调用BingoCage实例，从乱码的列表中取出一个元素
import random
class BingoCage:
    def __init__(self,items):
        self._items=list(items)
        random.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')#如果列表为空，抛出异常
        
    #bingo.pick()的快捷方式是bingo()
    def __call__(self, *args, **kwargs):
        return self.pick()

In [27]:
bingo = BingoCage(range(3))
bingo.pick()

2

In [28]:
bingo()

0

In [29]:
callable(bingo)

True

5.6 函数内省

In [30]:
dir(factoria)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

与用户定义的常规类一样，函数使用__dict__属性存储赋予它的用户属性。这相当于一种基本形式的注解。一般来说，为函数随意赋予属性不是很常见的做法，但django框架这么做了。

In [31]:
class C:pass
obj=C()
def func():pass
sorted(set(dir(func))-set(dir(obj)))#计算差集，然后排序，得到类的实例没有而函数有的属性类表。

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

5.7 从定位参数到仅限关键字参数

In [32]:
#tag函数用于生成HTML标签，使用名为cls的关键字参数传入“class”属性，这是一种变通方法，因为class是python的关键字

def tag(name,*content,cls=None,**attrs):
    '''生成一个或多个HTML标签'''
    if cls is not None:
        attrs['class']=cls
    if attrs:
        attr_str = ''.join(' %s="%s"'%(attr,value) for attr,value in sorted(attrs.items()))
    else:
        attr_str=''
    if content:
        return '\n'.join('<%s%s>%s</%s>'%
                         (name,attr_str,c,name)for c in content)
    else:
        return '<%s%s />'%(name,attr_str)

In [33]:
tag('br')

'<br />'

In [34]:
tag('p','hello')

'<p>hello</p>'

In [35]:
print(tag('p','hello','world'))

<p>hello</p>
<p>world</p>


In [36]:
tag('p','hello',id=33)

'<p id="33">hello</p>'

In [37]:
print(tag('p','hello','world',cls='sodebar'))#cls参数只能作为关键字参数传入

<p class="sodebar">hello</p>
<p class="sodebar">world</p>


In [38]:
tag(content='testing',name='img')#调用tag函数时，即使第一个定位参数也能作为关键字参数传入

'<img content="testing" />'

In [39]:
my_tag={'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg', 'cls': 'framed'}
tag(**my_tag)#在my_taf前加上**，字典中所有元素作为单个参数传入，同名键会绑定到对应的具名参数上，余下的则被**attrs捕获

'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

仅限关键字参数是python3新增的特性，在上例中，cls参数只能通过关键字参数指定，它一定不会捕获未命名的定位参数。定义函数时若想指定仅限关键字参数，要把它们放到前面的*的参数后面。

In [40]:
def f(a,*,b):
    return a,b

In [41]:
f(1,2)

TypeError: f() takes 1 positional argument but 2 were given

In [42]:
f(1,b=2)

(1, 2)

5.9 获取关于参数的信息

In [43]:
import bobo
@bobo.query('/')
def hello(person):
    return 'hello %s!'%person

bobo.query装饰器把一个普通的函数（如hello）与框架的请求处理机制集成起来了。Bobo会内省hello函数，发现它需要一个名为person的参数，然后从请求中获取那个名称对应的参数，将其传给hello函数，因此程序员根本不用触碰请求对象

启动脚本，bobo -f hello.py,访问http://localhost:8080/看到的消息是Missing form variable person,返回403

In [44]:
#示例5-15 在指定长度附近截断字符串的函数
def clip(text,max_len=80):
    end=None
    if len(text) > max_len:
        space_before = text.rfind(' ',0,max_len)
        if space_before >= 0:
            end=space_before
        else:
            space_after = text.rfind(' ',max_len)
            if space_after >= 0:
                end =space_after
    if end is None:
        end=len(text)
    return text[:end].rstrip()

函数对象有个__defaults__属性，它的值是一个元祖，里面保存着定位参数和关键字参数的默认值。仅限关键字参数的默认值在__kwdefaults__属性中。
然而，参数的名称在__code__属性中，它的值是一个code对象引用，自身也有很多属性

In [45]:
clip.__defaults__

(80,)

In [46]:
clip.__code__

<code object clip at 0x000001CABD8EA780, file "<ipython-input-44-3a960981e3a0>", line 2>

In [47]:
clip.__code__.co_varnames

('text', 'max_len', 'end', 'space_before', 'space_after')

In [48]:
clip.__code__.co_argcount#参数的个数，前2个，即text,和max_len

2

In [49]:
#提取函数的签名
from inspect import signature
sig = signature(clip)

In [50]:
sig

<Signature (text, max_len=80)>

In [51]:
str(sig)

'(text, max_len=80)'

In [52]:
for name ,param in sig.parameters.items():
    print(param.kind,':',name,'=',param.default)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


In [53]:
sig2=signature(sorted)
sig2

<Signature (iterable, key=None, reverse=False)>

In [54]:
for name ,param in sig2.parameters.items():
    print(param.kind,':',name,'=',param.default)#特殊的inspect._empty表示没默认值

POSITIONAL_OR_KEYWORD : iterable = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : key = None
POSITIONAL_OR_KEYWORD : reverse = False


![chapter5-8](image/chapter5-8.png)

In [55]:
#把tag函数的签名绑定到一个参数字典上
import inspect
sig = inspect.signature(tag)
bound_args = sig.bind(**my_tag)
bound_args

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>

In [56]:
del my_tag['cls']

In [57]:
bound_args=sig.bind(**my_tag)
bound_args

<BoundArguments (name='img', attrs={'src': 'sunset.jpg', 'title': 'Sunset Boulevard'})>

In [58]:
del my_tag['name']
bound_args=sig.bind(**my_tag)

TypeError: missing a required argument: 'name'

5.9 函数注解

In [59]:
def clip(text:str,max_len:'int > 0'=80) -> str: #有注解的函数声明
    end=None
    if len(text) > max_len:
        space_before = text.rfind(' ',0,max_len)
        if space_before >= 0:
            end=space_before
        else:
            space_after = text.rfind(' ',max_len)
            if space_after >= 0:
                end =space_after
    if end is None:#没找到空格
        end=len(text)
    return text[:end].rstrip()

注解不会做任何处理，只是在存储函数的__annotations__属性中

In [60]:
clip.__annotations__

{'max_len': 'int > 0', 'return': str, 'text': str}

In [61]:
sig=signature(clip)
sig.return_annotation

str

In [62]:
for param in sig.parameters.values():
    note=repr(param.annotation).ljust(13)
    print(note,':',param.name,'=',param.default)

<class 'str'> : text = <class 'inspect._empty'>
'int > 0'     : max_len = 80


5.10 支持函数式编程的包

5.10.1 operator模块

In [63]:
from functools import reduce
def fact(n):
    return reduce(lambda a,b:a*b,range(1,n+1))

operator模块为多个算术运算符提供了对应的函数，从而避免编写lambda a,b:a*b这种平凡的匿名函数

In [64]:
from functools import reduce
from  operator import mul
def fact(n):
    reduce(reduce(mul,range(1,n+1)))


operator中的itemgetter和attrgetter会自行构建函数

In [65]:
metro_data=[
    ('Tokyo','JP',36.933,(35.689722,1396.691667)),
    ('Delhi NCR','IN',21.935,(28.613889,77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
    
]

In [66]:
from operator import itemgetter
for city in sorted(metro_data,key=itemgetter(1)):#itemgetter(1)相当于lambda field:field[1]
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 1396.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


In [67]:
#如果把多个参数传给itemgetter，它构建的函数会返回提取的值构成的元组
cc_name = itemgetter(1,0)
for city in metro_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


attrgetter和itemgetter类似，它创建的函数根据名称提取对象的属性。如果把多个属性传给attrgetter，它也会返回提取的值构成的元组。此外，如果参数名中含.(点号)，attrgetter会深入嵌套对象，获得指定的属性

In [68]:
from collections import namedtuple
Latlong=namedtuple('LatLong','lat long')
Metropolis = namedtuple('Metropolis','name cc pop coord')
metro_areas = [Metropolis(name,cc,pop,Latlong(lat,long))
               for name,cc,pop,(lat,long) in metro_data]
metro_areas[0]

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=1396.691667))

In [69]:
metro_areas[0].coord.lat

35.689722

In [70]:
from operator import attrgetter
name_lat = attrgetter('name','coord.lat')
for city in sorted(metro_areas,key=attrgetter('coord.lat')):
    print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


下面是operator模块中定义的部分函数（省略了以_开头的名称，因为它们基本是实现的细节）

In [71]:
import operator
[name for name in dir(operator) if not name.startswith('_')]

['abs',
 'add',
 'and_',
 'attrgetter',
 'concat',
 'contains',
 'countOf',
 'delitem',
 'eq',
 'floordiv',
 'ge',
 'getitem',
 'gt',
 'iadd',
 'iand',
 'iconcat',
 'ifloordiv',
 'ilshift',
 'imatmul',
 'imod',
 'imul',
 'index',
 'indexOf',
 'inv',
 'invert',
 'ior',
 'ipow',
 'irshift',
 'is_',
 'is_not',
 'isub',
 'itemgetter',
 'itruediv',
 'ixor',
 'le',
 'length_hint',
 'lshift',
 'lt',
 'matmul',
 'methodcaller',
 'mod',
 'mul',
 'ne',
 'neg',
 'not_',
 'or_',
 'pos',
 'pow',
 'rshift',
 'setitem',
 'sub',
 'truediv',
 'truth',
 'xor']

In [72]:
from operator import methodcaller
s='The time has come'
upcase = methodcaller('upper')
upcase(s)

'THE TIME HAS COME'

In [73]:
hiphenate=methodcaller('replace',' ','_')
hiphenate(s)

'The_time_has_come'

上例中的第一个测试只是为了展示methorcaller的用法，如果想把str.upper当成函数使用，只需在str类上调用即可，并传入一个参数

In [74]:
str.upper(s)

'THE TIME HAS COME'

第二个测试表明，methorcaller还可以冻结某些参数，也就是部分应用，这与functools.partial函数作用类似

5.10.2使用functools.partial冻结参数

In [75]:
from  operator import mul
from functools import  partial
triple=partial(mul,3)
triple(7)

21

In [76]:
import unicodedata,functools
nfc=functools.partial(unicodedata.normalize,'NFC')
s1='café'
s2='cafe\u0301'
s1,s2

('café', 'café')

In [77]:
s1==s2

False

In [78]:
nfc(s1)==nfc(s2)

True